## PII

Personally Identifiable Information (PII) is information that can be used to identify, locate, or contact an individual. This includes information about a person, as well as information about a person's activities. In our libraries' systems, we have lots of PII about our patrons that go far beyond their names and addresses. This can include demographic information like age or gender, information in our logs like username or IP address, and, in an academic context, affiliations with various schools, departments, programs, etc. An immportant thing to note about PII is that it can be linked to form a more complete picture about an individual. Taken in isolation, a single piece of information may seem innocuous enough, but when combined with other available information about an individual, it can significantly further the process of identifying them.

To begin our look at PII within a library context, let's go ahead and re-read our patron data, remembering to use our ID column as the index and to parse DOB as dates. 

In [ ]:
# import the pandas module
import pandas as pd

# read our patron data
patrondata = pd.read_csv('patron.csv', index_col='ID', parse_dates=['DOB'])
# check our structure
patrondata.info()
# check the data
patrondata

## Challenge: PII
What PII can you identify within the patron data?

## De-Identification
De-identification is the process of removing identifying information from a dataset. This can be through complete or partial omission, or substituting a contrived value for the actual infortmation.

One of the most straightforward and effective techniques we can use to protect private data is to exclude it from the derived datasets we will use for reporting or analysis. Fortunately, pandas provides us with the tools we need to do just that. Let's go ahead and remove some of the PII-containing columns we know we won't need. Note that for now, we will keep some others, so we can make use of them for our analysis. 

In [ ]:
patrondata = patrondata.drop(columns=['First', 'Last', 'First', 'Address', 'City', 'State'])
patrondata.head()

## Aggregation & Re-identification

In [ ]:
patrondata.groupby(['College','Department','Major','Degree']).size().reset_index(name='groupsize')

We can see very easily that some of these groupings, unto themselves uniquely identify individuals within our dataset. While discarding the most specific of these, `Degree` would help from a privacy standpoint, that approach might not be acceptable for our needs. A compromise might be to group our `Degree` data into Doctoral and Masters degree levels (noting our dataset does not happen to include Undergraduate data). In the long run, it might be more efficient to make a change in how we Extract our data, but we can still acheive the result we need as a step in our Transformation. To do this, we first need a list of unique degrees. Pandas provides us with an elegant means of doing just that...

In [ ]:
degrees = patrondata.Degree.unique()
degrees

Now we can write a function to return the degree level, based on the degree.

In [ ]:
masters_list = ['M.S.', 'M.A.', 'Sc.M.', 'M.F.A.', 'M.S.E.', 'M.H.S.', 'MBE', 'M.S.P.H.']
doctoral_list = ['Ph.D.', 'Dr.P.H.', 'Sc.D.']
def degree_level(degree):
    if degree in masters_list:
        return 'masters'
    elif degree in doctoral_list:
        return 'doctoral'
    else:
        return ''

As an aside, we can note a few things about this approach. The first is that we make a point of defining our lists outside of our function. This is done for efficiency, so that our lists don't get re-created each time our function is called. Second, those of us who are coming to Python from other programming languages might be inclined to use a case or switch statement for this purpose. You may be surprised to learn that Python has no such construct. While a variety of workarounds are possible, the sequence of `if... elif... elif... else` is [recommended in the docs](https://docs.python.org/3/faq/design.html#why-isn-t-there-a-switch-or-case-statement-in-python), and therefore the most pythonic solution.

Now we can use our function to populate a new `Degree_Level` column in our dataframe...

In [ ]:
patrondata['Degree_Level'] = patrondata['Degree'].apply(degree_level)
patrondata

And a quick check to make sure we didn't miss anything...

In [ ]:
missing = (patrondata['Degree_Level'] == '').sum()
print(missing)

And now we can repeat our grouping using our new column...

In [ ]:
patrondata.groupby(['College','Department','Major','Degree_Level']).size().reset_index(name='groupsize')

This is a slight improvement, but we may need to strike a different balance between the specificity of our data and our patron's privacy. Let's look at some other options.

We could exclude the next step up in our heirarchy, `Major`...

In [ ]:
patrondata.groupby(['College','Department','Degree_Level']).size().reset_index(name='groupsize')

This is much better, but still contains some problematically small groupings, and there are still lots of 1's in this list. For the sake of it, let's see what our results might look like with only `College` and `Department`...

In [ ]:
patrondata.groupby(['College','Department']).size().reset_index(name='groupsize')

Again, much improved, but now looking at some of our smallest groupings, we can more easily notice a problem in our data there are a couple of `Department` entries that are clearly meant to be the same, but that were apparently keyed in differently at some point along the line. Not only does this skew our data, but it also constitutes a privacy risk. Anomalous values can make re-identification much easier, and in some cases can act like a fingerprint, uniquely identifying individuals who might otherwise be protected by aggregation. Again, we should find the source of these issues in our data collection systems or our Extraction processes, but for now, we can address them here...

In [ ]:
patrondata['Department'] = patrondata['Department'].replace('Population, Family, and Reproductive Health', 'Population, Family and Reproductive Health')
patrondata['Department'] = patrondata['Department'].replace('German & Romance Languages & Literatures','German and Romance Languages and Literatures')
patrondata.groupby(['College','Department']).size().reset_index(name='groupsize')

Better, but _how much_ better? And, are there any problematic counts hiding in that break between our first and last rows? Again, pandas helps us sort this out...

In [ ]:
patrondata.groupby(['College','Department']).size().reset_index(name='groupsize').sort_values('groupsize')

Ok, so some of these `Departments` are simply small, no matter how we slice it... Perhaps it would be better to create some arbitrary groups to get all of these counts into double-digits. To do this, we can use a similar function to our `degree_levels`. First, we'll need to filter for those small departments...
FIXME: cite how big from [PPP](https://rowman.com/ISBN/9781442269699/Protecting-Patron-Privacy-A-LITA-Guide) pg 51

In [ ]:
departmentdata = patrondata.groupby(['College','Department']).size().reset_index(name='groupsize')
departmentdata[departmentdata.groupsize < 10]

Now we can put those `Departments` into a couple of lists, based on `College`.

In [ ]:
som_small_departments = departmentdata[(departmentdata.groupsize < 10) & (departmentdata.College == 'School of Medicine')].Department.tolist()
som_small_departments

In [ ]:
as_small_departments = departmentdata[(departmentdata.groupsize < 10) & (departmentdata.College == 'School of Arts and Sciences')].Department.tolist()
as_small_departments

Now we're ready for that function...

In [ ]:
def small_departments(department):
    if department in som_small_departments:
        return ', '.join(som_small_departments)
    elif department in as_small_departments:
        return ', '.join(as_small_departments)
    else:
        return department

In [ ]:
departmentdata['Department'] = departmentdata['Department'].apply(small_departments)
departmentdata

We can see our new small department labels in the results, but the row count didn't change. What happened?  
Well, the labels are standing in for the actual `Department` names, but the rows are otherwise unchanged. In order to see the impact in that way, we will need to roll those newly equavalent values up and reindex the DataFrame, as follows:

In [ ]:
departmentdatagrouped = departmentdata.groupby(['College','Department']).sum()
departmentdatagrouped.reset_index()

Some of these group counts are still smaller than we might hope for, and this is simply how it is with some data. Ultimately, it will be up to you, your colleagues, and your patrons to work out the your balance between having data upon which to make decisions and protecting your patrons' privacy. Using what we already know, we could construct more groupings to use a safer threshold for aggregation, or we could decide that `Department` is just just too fine a grouping for our data, and report affiliation only at the level of `College`. As an exercise, let's see what that would look like. Try creating a set of college data from our patron data. Include a groupsize, and sort on that value.

In [ ]:
collegedata = patrondata.groupby(___).size().reset_index(___).sort_values(___)
collegedata

While admittedly less granular, this grouping clearly constitutes less risk of reidentification. So much so, that we might go back to our `Degree_Level` and see what a grouping of `College` and `Degree_Level` might look like. Let's re-build our college data in that way, by reassigning our `collegedata` variable. Remember that we already applied our `degree_level` function to our patron data and incorporated it into our dataset, so we don't have to do that again here.

In [ ]:
collegedata = patrondata.groupby(['College','Degree_Level']).size().reset_index(name='groupsize').sort_values(['College','groupsize'], ascending=False)
collegedata

Great! While these group counts are still not huge, they are far better than what we started with. Now we know that in our analysis of our circulation data, if we report, for example, a checkout by a masters student at the School of Arts and Sciences, we will be referring to one of 84 students; as opposed to describing a checkout by that single Human Genetics and Molecular Biology major pursuing their M.S. in the Biology Department.

Before we move on from our patron data, let's take one more look for risks that we might mitigate.

In [ ]:
patrondata

Clearly, we will want to do something about the `Patron_ID`, but we will need it for matching up our circulation data, so we will need to leave it in place for now. See the discussion section for some ideas about ZIP Codes, but they too can stick around for now. `DOB`, or date-of-birth, on the other hand, looks worth some of our time. While not _necessarily_ unique, we know without doing a lot of analysis that `DOB` will be enough to identify many of the individuals within our data. A better substitute would be age. Let's create a function for calculating age from date-of-birth

In [ ]:
from datetime import date
def calculate_age(dob):
    today = date.today()
    return today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))

### Callout: Python imports revisited
We're doing something slightly different here as we import the datetime module from the standard library. Rather than just `import datetime` (which would certainly work), we're specifically importing the `date` class from the `datetime` module which defines it. Not only does this keep our notebook (or program) slightly smaller by not adding additional classes we don't need, it also makes our code a little simplier and easier to read as we can now just refer to our `date` class by name, rather than needing to refer to it as `datetime.date`. Either way, it brings us its `year`, `month`, and `day` attributes, as well as its method `today()`, all of which we make use of here. Finally, the eagle-eyed among us might notice that while we were explicit about creating our `today` variable as a `date` to get its `year`, `month`, and `day`; we manage to get the same attributes from our passed-in `dob` variable. This is because we instructed pandas to parse our `DOB` values as dates when we first imported our date, as a result, they are stored as `datetime64` objects, a compatible class that pandas gets from the numpy package, that it builds upon. The `datetime64` class also has `year`, `month`, and `day` attributes that we can make use of in the same way.

Now that we have our `calculate_age` function, let's put it to use to populate a new `Age` column.

In [ ]:
patrondata['Age'] = patrondata['DOB'].apply(calculate_age)
patrondata

Now let's see how big our group sizes would be if we grouped by `Age`...

In [ ]:
patrondata.groupby(['Age']).size().reset_index(name='groupsize').sort_values('groupsize')

Most of these look fine, but unfortunately, or data happens to have a noticeably small group of 59-year-olds. Perhaps instead of `Age`, what we need is an `Age_Range`. Lets write a new function for calculating that, building on our `calculate_age` function.

In [ ]:
from datetime import date
def calculate_age_range(dob):
    today = date.today()
    age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
    min = age - (age % 5)
    max = min + 5
    range = '{:d} - {:d}'.format(min,max)
    return range

Now apply it, to populate a new `Age_Range` column...

In [ ]:
patrondata['Age_Range'] = patrondata['DOB'].apply(calculate_age_range)
patrondata

And finally, check our group sizes...

In [ ]:
patrondata.groupby(['Age_Range']).size().reset_index(name='groupsize').sort_values('groupsize')

Very nice. Those are some very safe looking aggregates. We'll have to rebuild our patron data again when we are ready to merge it with our circulation data, so we needn't drop our `DOB` and `Age` columns just now, but I think we have a pretty good plan for what we will do when we get there. In the meantime, let's check out our circulation data.